In [1]:
import pandas as pd
import datetime
from datetime import timedelta
import requests
import plotly as py
import plotly.express as px
import chart_studio.tools as tls

In [2]:
#BUILDS DATAFRAME FOR COMMUNITY DATA
columns = ['id', 'symbol', 'price', 'mkt_cap', 'vol', 'reddit_posts_48h', 'reddit_comments_48h', 'reddit accounts_48h', 'forks', 'stars', 'contributors', 'twitter_followers', 'alexa_rank']
df = pd.DataFrame(columns = columns)
# display(df)
today = datetime.datetime.now()
yesterday = today -timedelta(days = 2)
yesterday = yesterday.strftime("%d-%m-%Y")
# date = str(yesterday)
date = yesterday
# print('yesterday', yesterday)
# print('date', date)

def coinFunc(currentCoin, date): 
    coin= currentCoin
    geckoReq = 'https://api.coingecko.com/api/v3/coins/'+coin+'/history?date='+date
    priceReq = 'https://api.coingecko.com/api/v3/simple/price?ids=' + coin + '&vs_currencies=usd&include_market_cap=true&include_24hr_vol=true'
    price = requests.get(priceReq).json()
    r =requests.get(geckoReq).json()
#     display(r)
    #     print(r)
    coin_id = r['id']
    coin_symbol = r['symbol']

    #price data
    current_price = price[currentCoin]['usd']
    mkt_cap = price[currentCoin]['usd_market_cap']
    vol = price[currentCoin]['usd_24h_vol']

    #reddit data
    reddit_average_posts_48h = r['community_data']['reddit_average_posts_48h']
    reddit_average_comments_48h = r['community_data']['reddit_average_comments_48h']
    reddit_accounts_active_48h = r['community_data']['reddit_accounts_active_48h']
    #github data
    forks = r['developer_data']['forks']
    stars = r['developer_data']['stars']
    contributors = r['developer_data']['pull_request_contributors']
    twitter_followers = r['community_data']['twitter_followers']
    alexa_rank = r['public_interest_stats']['alexa_rank']


    dataInsert = [coin_id, coin_symbol, current_price, mkt_cap, vol, reddit_average_posts_48h, reddit_average_comments_48h, reddit_accounts_active_48h, forks, stars, contributors, twitter_followers, alexa_rank]
    # print(dataInsert)

    df_length = len(df)
    df.loc[df_length] = dataInsert
    
# coinFunc('audius', yesterday)    
# display(df)
# print(r)
# print(r['developer_data'])
# print(r['community_data'])
# print(coin_price)

theCoins = ['ethereum', 'bitcoin', 'binancecoin', 'solana', 'polkadot', 'flow', 'dogecoin', 'audius']
trendingReq = 'https://api.coingecko.com/api/v3/search/trending'
trends =requests.get(trendingReq).json()
# print(r)
trending = []
#could be for 7 coins but lets just do top 5
for i in range(5):
    trendingCoin = trends['coins'][i]['item']['id']
    trending.append(trendingCoin)

theCoins.extend(trending)

for crypto in theCoins:
    coinFunc(crypto, yesterday)
    
display(df)
myCoinsList = df['id'].tolist()
print(myCoinsList)

,id,symbol,price,mkt_cap,vol,reddit_posts_48h,reddit_comments_48h,reddit accounts_48h,forks,stars,contributors,twitter_followers,alexa_rank
0,ethereum,eth,4.514970e+03,5.341573e+11,2.371612e+10,7.091,346.818,1308.08333333333,12476,33739,595,1907100,3578
1,bitcoin,btc,5.703700e+04,1.075281e+12,3.910990e+10,6.000,907.364,3659.0,30685,59661,760,3871003,6384
2,binancecoin,bnb,6.223100e+02,1.044446e+11,2.605239e+09,9.727,178.455,818.416666666667,None,None,None,6376270,123
3,solana,sol,2.360300e+02,7.203360e+10,3.956357e+09,6.667,113.250,460.307692307692,1130,5823,209,1034232,3672
4,polkadot,dot,3.615000e+01,3.830617e+10,9.722963e+08,2.273,35.182,91.3333333333333,1008,4797,141,None,15201
5,flow,flow,1.206000e+01,3.791135e+09,8.482743e+07,0.000,0.000,None,None,None,None,None,329714
6,dogecoin,doge,2.098790e-01,2.773102e+10,1.926677e+09,8.091,732.364,2521.08333333333,2378,13518,126,2584497,32279
7,audius,audio,2.280000e+00,1.161846e+09,3.644907e+07,1.273,13.273,21.5833333333333,None,None,None,None,20469
8,rainbowtoken,rainbowtoken,3.294000e-08,2.640280e+07,6.769669e+06,0.000,0.000,None,0,0,0,None,141159
9,urus-token,urus,2.166900e+02,1.257989e+08,1.297532e+06,0.000,0.000,None,None,None,None,None,47272


['ethereum', 'bitcoin', 'binancecoin', 'solana', 'polkadot', 'flow', 'dogecoin', 'audius', 'rainbowtoken', 'urus-token', 'hoge-finance', 'lcx', 'ufo-gaming']


In [3]:
### BUILDING REDDIT COMMENTS HTML GRAPH
display(df.head(5))

fig = px.histogram(df, x=df['id'], y=df['reddit_comments_48h'],
                labels={
                     "sum of reddit_comments_48h": "Reddit Comments 48h",
                     "id": "token"
#                      "species": "Species of Iris"
                 },)
py.offline.plot(fig, filename='geckoRedditComments.html')


# print(tls.get_embed('https://plotly.com/~fooba/1'))
fig.show()



,id,symbol,price,mkt_cap,vol,reddit_posts_48h,reddit_comments_48h,reddit accounts_48h,forks,stars,contributors,twitter_followers,alexa_rank
0,ethereum,eth,4514.97,5.341573e+11,2.371612e+10,7.091,346.818,1308.08333333333,12476,33739,595,1907100,3578
1,bitcoin,btc,57037.00,1.075281e+12,3.910990e+10,6.000,907.364,3659.0,30685,59661,760,3871003,6384
2,binancecoin,bnb,622.31,1.044446e+11,2.605239e+09,9.727,178.455,818.416666666667,None,None,None,6376270,123
3,solana,sol,236.03,7.203360e+10,3.956357e+09,6.667,113.250,460.307692307692,1130,5823,209,1034232,3672
4,polkadot,dot,36.15,3.830617e+10,9.722963e+08,2.273,35.182,91.3333333333333,1008,4797,141,None,15201


In [4]:
#clean version of historic prices request
#timestamps are in day-month-year
import requests
import datetime
import pandas as pd
from datetime import timedelta

# dfPrices = pd.DataFrame()
# geckoReq = 'https://api.coingecko.com/api/v3/coins/ethereum/market_chart?vs_currency=usd&days=60&interval=daily'
geckoReq = 'https://api.coingecko.com/api/v3/coins/ethereum/market_chart?vs_currency=usd&days=60&interval=daily'

r = requests.get(geckoReq).json()
ts = r['prices'][0][0]
ts = ts/1000
HistPricesList = []
for i in range(len(r['prices'])):
    currentUnix = r['prices'][i][0] 
    price = r['prices'][i][1]
    currentUnix = currentUnix/1000
    currentTS = datetime.datetime.fromtimestamp(currentUnix).strftime("%d-%m-%Y")
    HistPricesList.append([currentTS])
global dfPrices
dfPrices = pd.DataFrame(HistPricesList, columns = ['dates'])
print(dfPrices)

# display(dfCoin)
# print('dfprices')
# display(dfPrices)

# coin='ethereum'
def historicalCoinPrices(coin):
    global dfPrices
    # geckoReq = 'https://api.coingecko.com/api/v3/coins/ethereum/market_chart?vs_currency=usd&days=60&interval=daily'
    geckoReq = 'https://api.coingecko.com/api/v3/coins/'+coin+'/market_chart?vs_currency=usd&days=60&interval=daily'

    r = requests.get(geckoReq).json()
    ts = r['prices'][0][0]
    # print(ts)
    ts = ts/1000
    print(datetime.datetime.fromtimestamp(ts).strftime("%d-%m-%Y"))
    # print('prices length',len(r['prices']))
    HistPricesList = []
    for i in range(len(r['prices'])):
        currentUnix = r['prices'][i][0] 
        price = r['prices'][i][1]
        currentUnix = currentUnix/1000
        currentTS = datetime.datetime.fromtimestamp(currentUnix).strftime("%d-%m-%Y")
    #     print('price: ', price, 'TS: ', currentTS)
        HistPricesList.append([currentTS, price])
    #     currentUnix = currentTS
#     print(HistPricesList)
    dfCoin = pd.DataFrame(HistPricesList, columns = ['dates', coin])
    # display(dfCoin)
    dfPrices = pd.merge(dfPrices, dfCoin[coin], left_on = dfPrices['dates'], right_on=dfCoin['dates']).drop(['key_0'], axis = 1)
    # display(dfPrices)

for i in myCoinsList:
    historicalCoinPrices(i)
    print('just added: ', i)


display(dfPrices.head(5))

         dates
0   03-10-2021
1   04-10-2021
2   05-10-2021
3   06-10-2021
4   07-10-2021
..         ...
56  28-11-2021
57  29-11-2021
58  30-11-2021
59  01-12-2021
60  02-12-2021

[61 rows x 1 columns]
03-10-2021
just added:  ethereum
03-10-2021
just added:  bitcoin
03-10-2021
just added:  binancecoin
03-10-2021
just added:  solana
03-10-2021
just added:  polkadot
03-10-2021
just added:  flow
03-10-2021
just added:  dogecoin
03-10-2021
just added:  audius
03-10-2021
just added:  rainbowtoken
03-10-2021
just added:  urus-token
03-10-2021
just added:  hoge-finance
03-10-2021
just added:  lcx
03-10-2021
just added:  ufo-gaming


,dates,ethereum,bitcoin,binancecoin,solana,polkadot,flow,dogecoin,audius,rainbowtoken,urus-token,hoge-finance,lcx,ufo-gaming
0,03-10-2021,3426.387081,48282.971139,431.449016,172.855870,32.200487,19.104391,0.222176,2.381418,6.653215e-09,91.467224,0.000093,0.165840,0.000006
1,04-10-2021,3390.310408,49260.232321,427.545862,167.877910,31.185246,20.613104,0.243219,2.249091,6.159209e-09,90.337881,0.000089,0.176603,0.000007
2,05-10-2021,3520.342257,51668.994224,443.435448,164.369276,31.541054,20.535903,0.251694,2.210754,7.998785e-09,92.118116,0.000118,0.182719,0.000008
3,06-10-2021,3592.176130,55415.005742,435.262006,154.094624,32.485706,19.171906,0.252551,2.154357,5.988382e-09,96.875135,0.000126,0.188018,0.000007
4,07-10-2021,3594.918358,53894.595995,439.932647,154.259885,33.831460,19.432543,0.243439,2.113511,4.759678e-09,96.083633,0.000113,0.153320,0.000007


In [5]:
#build price percent Changes Graph
dfPercents = dfPrices
dfPercents = dfPercents.set_index('dates')
dfPercents = dfPercents.pct_change()
display(dfPercents.head(5))
#the default y axis label was value so thats why it is being changed below
fig = px.line(dfPercents, labels = {'value': 'Percent Change', 'variable': 'coin'})
py.offline.plot(fig, filename='pricePercentChangesLines.html')

fig.show()


,ethereum,bitcoin,binancecoin,solana,polkadot,flow,dogecoin,audius,rainbowtoken,urus-token,hoge-finance,lcx,ufo-gaming
dates,,,,,,,,,,,,,
03-10-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
04-10-2021,-0.010529,0.020240,-0.009047,-0.028798,-0.031529,0.078972,0.094714,-0.055567,-0.074251,-0.012347,-0.050685,0.064903,0.127672
05-10-2021,0.038354,0.048899,0.037165,-0.020900,0.011410,-0.003745,0.034844,-0.017045,0.298671,0.019706,0.335919,0.034627,0.127088
06-10-2021,0.020405,0.072500,-0.018432,-0.062510,0.029950,-0.066420,0.003403,-0.025510,-0.251339,0.051640,0.066035,0.029004,-0.112465
07-10-2021,0.000763,-0.027437,0.010731,0.001072,0.041426,0.013595,-0.036079,-0.018960,-0.205181,-0.008170,-0.102611,-0.184549,-0.078832
